In [1]:
!pip install langchain langchain-openai

# 多次调用完成中译英，英文文答，英译中

In [1]:
import os
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI


llm = ChatOpenAI(model_name="gpt-3.5-turbo", max_tokens=2048, temperature=0.5)

en_to_zh_prompt = PromptTemplate(
     template="请把下面这句话翻译成英文： \n\n {question}?", input_variables=["question"]
)

question_prompt = PromptTemplate(
    template = "{english_question}", input_variables=["english_question"]
)

zh_to_cn_prompt = PromptTemplate(
    input_variables=["english_answer"],
    template="请把下面这一段翻译成中文： \n\n{english_answer}?",
)

question_translate_chain = LLMChain(llm=llm, prompt=en_to_zh_prompt, output_key="english_question")
english = question_translate_chain.run(question="请你作为一个机器学习的专家，介绍一下CNN的原理。")
print(english)

qa_chain = LLMChain(llm=llm, prompt=question_prompt, output_key="english_answer")
english_answer = qa_chain.run(english_question=english)
print(english_answer)

answer_translate_chain = LLMChain(llm=llm, prompt=zh_to_cn_prompt)
answer = answer_translate_chain.run(english_answer=english_answer)
print(answer)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


As an expert in machine learning, could you please introduce the principles of CNN?
Certainly! Convolutional Neural Networks (CNNs) are a type of deep learning model that is commonly used for image recognition and computer vision tasks. The key principles of CNNs include:

1. Convolutional layers: CNNs use convolutional layers to extract features from the input image. These layers apply a set of filters (also known as kernels) to the input image to detect patterns and features such as edges, textures, and shapes.

2. Pooling layers: Pooling layers are used to downsample the feature maps generated by the convolutional layers. This helps reduce the spatial dimensions of the feature maps and makes the network more computationally efficient.

3. Activation functions: Activation functions such as ReLU (Rectified Linear Unit) are used to introduce non-linearity into the network, allowing it to learn complex patterns and relationships in the data.

4. Fully connected layers: After the convolu

# 使用Langchain简化实现

In [3]:
from langchain.chains import SimpleSequentialChain

chinese_qa_chain = SimpleSequentialChain(
    chains=[question_translate_chain, qa_chain, answer_translate_chain],
    input_key="question",
    verbose=True,
)
answer = chinese_qa_chain.run(question="请你作为一个机器学习的专家，介绍一下CNN的原理。")
print(answer)



> Entering new SimpleSequentialChain chain...
As an expert in machine learning, could you please introduce the principles of CNN?
Certainly! Convolutional Neural Networks (CNNs) are a type of deep learning algorithm that is commonly used for image recognition and classification tasks. The key principles of CNNs include:

1. Convolutional layers: CNNs use convolutional layers to extract features from input images. These layers apply filters (also known as kernels) to the input image, which helps the network detect patterns and features such as edges, textures, and shapes.

2. Pooling layers: Pooling layers are used to reduce the spatial dimensions of the feature maps generated by the convolutional layers. This helps to make the network more robust to variations in the input data and reduces the computational complexity of the model.

3. Activation functions: Activation functions introduce non-linearity into the network, allowing it to learn complex patterns and relationships in the da

# 多步骤引入不同输入参数

In [4]:
from langchain.chains import SequentialChain

q1_prompt = PromptTemplate(
    input_variables=["year1"],
    template="{year1}年的欧冠联赛的冠军是哪支球队，只说球队名称。"
)
q2_prompt = PromptTemplate(
    input_variables=["year2"],
    template="{year2}年的欧冠联赛的冠军是哪支球队，只说球队名称。"
)
q3_prompt = PromptTemplate(
    input_variables=["team1", "team2"],
    template="{team1}和{team2}哪只球队获得欧冠的次数多一些？"
)
chain1 = LLMChain(llm=llm, prompt=q1_prompt, output_key="team1")
chain2 = LLMChain(llm=llm, prompt=q2_prompt, output_key="team2")
chain3 = LLMChain(llm=llm, prompt=q3_prompt)

sequential_chain = SequentialChain(chains=[chain1, chain2, chain3], input_variables=["year1", "year2"], verbose=True)
answer = sequential_chain.run(year1=2000, year2=2010)
print(answer)



> Entering new SequentialChain chain...

> Finished chain.
皇家马德里队获得欧冠的次数更多，他们共获得了13次欧冠冠军，而国际米兰队获得了3次欧冠冠军。


# 通过Langchain来完成自动撰写单元测试

In [19]:
from langchain_openai import OpenAI
from langchain.chains import SequentialChain

def write_unit_test(function_to_test, unit_test_package = "pytest"):
    # 解释源代码的步骤
    explain_code = """"# How to write great unit tests with {unit_test_package}

    In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
    ```python
    {function_to_test}
    ```

    Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
    - First,"""

    explain_code_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test"],
        template=explain_code
    )
    explain_code_llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.4, max_tokens=1000,
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    explain_code_step = LLMChain(llm=explain_code_llm, prompt=explain_code_template, output_key="code_explaination")

    # 创建测试计划示例的步骤
    test_plan = """

    A good unit test suite should aim to:
    - Test the function's behavior for a wide range of possible inputs
    - Test edge cases that the author may not have foreseen
    - Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
    - Be easy to read and understand, with clean code and descriptive names
    - Be deterministic, so that the tests always pass or fail in the same way

    `{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

    For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
    -"""
    test_plan_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination"],
        template= explain_code + "{code_explaination}" + test_plan
    )
    test_plan_llm = OpenAI(model_name="gpt-3.5-turbo-instruct", max_tokens=1000, temperature=0.5,
            top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"])
    test_plan_step = LLMChain(llm=test_plan_llm, prompt=test_plan_template, output_key="test_plan")

    # 撰写测试代码的步骤
    starter_comment = "Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator"
    prompt_to_generate_the_unit_test = """

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

#{starter_comment}"""

    unit_test_template = PromptTemplate(
        input_variables=["unit_test_package", "function_to_test", "code_explaination", "test_plan", "starter_comment"],
        template= explain_code + "{code_explaination}" + test_plan + "{test_plan}" + prompt_to_generate_the_unit_test
    )
    unit_test_llm = OpenAI(model_name="gpt-3.5-turbo-instruct", temperature=0.4, max_tokens=1000, stop="```")
    unit_test_step = LLMChain(llm=unit_test_llm, prompt=unit_test_template, output_key="unit_test")

    sequential_chain = SequentialChain(chains=[explain_code_step, test_plan_step, unit_test_step],
                                    input_variables=["unit_test_package", "function_to_test", "starter_comment"], verbose=True)
    answer = sequential_chain.run(unit_test_package=unit_test_package, function_to_test=function_to_test, starter_comment=starter_comment)
    return f"""#{starter_comment}""" + answer

code = """
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
"""

import ast

def write_unit_test_automatically(code, retry=3):
    unit_test_code = write_unit_test(code)
    all_code = code + unit_test_code
    tried = 0
    while tried < retry:
        try:
            ast.parse(all_code)
            return all_code
        except SyntaxError as e:
            print(f"Syntax error in generated code: {e}")
            all_code = code + write_unit_test(code)
            tried += 1

print(write_unit_test_automatically(code))

/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_core/utils/utils.py:159: UserWarning: WARNING! stop is not default parameter.
                stop was transferred to model_kwargs.
                Please confirm that stop is what you intended.
  warnings.warn(




> Entering new SequentialChain chain...

> Finished chain.

def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)
    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
#Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator
#The first element of the tuple is the input to the function and the second element is the expected output
@pytest.mark.parametrize(
    "input, expected_output",
    [
        (3600, "1h0min0s"),
        (3665, "1h1min5s"),
        (7200, "2h0min0s"),
        (120, "2min0s"),
        (125, "2min5s"),
        (360, "6min0s"),
        (10, "10s"),
        (30, "30s"),
        (59, "59s"),
        (-1, None),  # we expect an error for negative inputs
        ("abc", None),  # we expect an error for non-numeric inputs
        (None, None),  # we expect an er

In [5]:
!pip list

Package                                 Version
--------------------------------------- --------------
aiofiles                                22.1.0
aiohttp                                 3.9.5
aiosignal                               1.3.1
aiosqlite                               0.20.0
altair                                  5.3.0
annotated-types                         0.6.0
anyio                                   4.3.0
argon2-cffi                             23.1.0
argon2-cffi-bindings                    21.2.0
arrow                                   1.3.0
astroid                                 3.1.0
asttokens                               2.4.1
async-timeout                           4.0.3
attrs                                   23.2.0
autopep8                                2.0.4
Babel                                   2.14.0
backoff                                 2.2.1
beautifulsoup4                          4.12.3
bleach                                  6.1.0
blis            